# TODO: Cambiar nauyaca_local a nauyaca

# 1.- Introduction

nauyaca: Bothrops asper: (México; from Nahuatl nahui, four, and yacatl, nose; "four noses")

## Overview

Nauyaca is a python package exclusively adapted to find planetary masses and orbits of transiting planets, through the fitting of mid-transit times, from an N-body approach. 

The tool manages the communication with TTVFast (Deck et al., 2014) and many minimization algorithms and a parallel-tempered MCMC (ptemcee; Vousden et al. 2015).

To work corrctly, Nauyaca requieres measured transit time ephemerids of the planet(s), composed by the number of the transit (starting from epoch zero at a given reference epoch T0), the central transit time, and the proper lower and upper errors. Additionally the stellar mass and radius are requiered.

Nauyaca implements minimization algorithms, namely Differential Evolution, Powell and Nelder-Mead. It also allows to perform a Markov chain Monte Carlo method to fully explore the planetary parameter space. As a result, Nauyaca returns a posteriori distributions of the planetary orbital elements and masses. A set of predefined figures allow to examine the performance and take decisions about the results.

In this tutorial we will learn to use Nauyaca to solve a synthetic two-planet system.

Let's start!
_____________________________________

## Installation 

Python 3.7 is requiered

clone the repository in a new workspace, for example:

mkdir NAU

cd NAU

git clone https://github.com/EliabCanul/nauyaca.git

EliabCanul

#####

Install the dependencies:

pipenv install


Activate the virtual environment:

pipenv shell

## 1.1 Example: Solving for a two-planet system

Let's import the nauyaca, given the alias: nau

In [1]:
import nauyaca_local as nau

We inspect the available modules inside nauyaca:

In [2]:
nau.__all__

['SetPlanet', 'PlanetarySystem', 'Optimizers', 'MCMC', 'Plots_c']

### 1.1.1 The Planet object

First, let's define the Planet objects and their attributes. These planets will be part of the planetary system that we want to analyze. We create a Planet just by giving a planet name. Let's assume that the first planet in our system is named simply "Planet-b". Then, we create it as follows:

In [3]:
Planetb = nau.SetPlanet("Planet-b")

Nice! it was so easy. What can we do now? Well, since we want to fit the mid-transit times of our planet, let's give it the transit data.

#### Transit times data

There are two ways of giving the mid-transit time observations to the Planet object. 

**1.-** The first one is by explicity giving a dictionary of the ephemerids to the attribute -ttvs_data-, in the following format:

In [4]:
Planetb.ttvs_data = {0: [1.941979, 0.000144, 0.000144],
                     1: [7.296786, 0.000283, 0.000283],
                     2: [12.654408,0.000314, 0.000314]
                     #... more data
                    }

where the keys are the transit epochs (integer numbers) and the values are lists containing, for each epoch, the central transit time and the lower and upper errors, in days.

**2.-** The second one, is by using the .load_ttvs method. When using this way, the ephemerid data should be in an ascci file containing the transit epochs, the central transit times and the lower and upper errors, as before.

For example, a file containing these data looks like:

In [5]:
! head -8  ./inputs/planetb_ttvs.dat

#Transit  t_c           lower_err      upper_err
#Number  (days)          (days)          (days) 
0	1.941979	0.000144	0.000144
1	7.296786	0.000283	0.000283
2	12.654408	0.000314	0.000314
3	18.010997	0.000356	0.000356
4	23.366161	0.000336	0.000336
5	28.721616	0.000361	0.000361


**Note:** the transit epochs must always start from 0 at least for one planet. It means that epochs must be labeled from 0 at a reference epoch.

Thus, for ease we will use the second way. Hence, we load the transit data as follows:

In [5]:
Planetb.load_ttvs("./inputs/planetb_ttvs.dat")  # Note that the data are in the inputs subdirectory

Fine! at this point we have a Planet with transit data. Now, we will see how to say to Nauyaca where to search in the parameter space of our planet...

#### Setting the parameter space

For a planet, we have 7 parameters that Nauyaca will explore (and the used units):

* Mass    -----  [M_earth]
* Period  ----- [days]
* Eccentricity
* Inclination  ----- [deg]
* Argument of periastron  ----- [deg]
* Mean anomaly  ----- [deg]
* Ascending node  ----- [deg]


Now, in order to avoid exploring in a non-physical regions of the parameter space or with redundant information, it is necessary to establish especific boundaries for each planetary parameter. These boundaries will be used in the subsequent analysis. 

However, if nothing is known about one or more parameters, a $default$ set of boundaries will be established. Let's see which are these default boundaries by calling the physical limits established inside the constants of Nauyaca:

In [7]:
nau.constants.physical_bounds

{'mass': (0.0123032, 25426.27252757),
 'period': (0.1, 1000.0),
 'ecc': (1e-06, 0.9),
 'inclination': (0.0, 180.0),
 'argument': (0.0, 360),
 'mean_anomaly': (0.0, 360),
 'ascending_node': (0.0, 360.0)}

By default the mass is ranged from a lunar mass up to 80 Jupiter masses (units above are in Earth masses). Angles are restricted bewteen 0° and 360° except for the inclination which is defined between 0° and 180°. These default boundaries arise if anything is known about the planet. In practice we can restrict the parameter space from previous knowledge coming, for example, from light curve analysis. 

To restrict any planetary parameter, we should give a range [minimum, maximum]. Those parameters that we do not restrict manually will have the default limits mentioned above.

For our example, let's suppose that we know a rough estimation of the period and the inclination. It is true because we can approximate the period from consecutive transit times, and also we know that the planet transits, so the inclination should be around ~90°! Let's also suppose that we want to explore the mass range from a sub-Earth planet to a Neptune. Finally, let's the eccentricity to vary up to 0.2, because greater values would result in unstable orbits.

Thus, to make these restrictions, we write:

In [6]:
Planetb.mass = [.1, 20.]             # Planet mass [Mearth]
Planetb.period = [5.34, 5.36]         # Orbital period [days]
Planetb.ecc = [0., 0.2]                # eccentricity. Look at the lower limit!
Planetb.inclination = [85, 95]         # Orbital inclination [deg]

print(Planetb)  # print a summary of our Planet object


 =========== Planet Summary =========== 
--> Parameter -ecc- of planet -Planet-b- set to lower physical limit: 1e-06
Planet : Planet-b
  Boundaries:
    mass: (0.1, 20.0)  [M_earth]
    period: (5.34, 5.36)  [d]
    ecc: (1e-06, 0.2)   
    inclination: (85, 95)  [deg]
    argument: (0.0, 360)  [deg]
    mean_anomaly: (0.0, 360)  [deg]
    ascending_node: (0.0, 360.0)  [deg]
  TTVs: True


From here, we can notice of two things:

**1.-** There is a dialog from Nauyaca (starting with -->) that say tu us that the lower boundary of eccentricity have been set to 1e-06. Why is that? It is because we manually set the eccentricity lower limit to 0, but this value is lower than the default lower limit in eccentricity mentioned previously: 1e-0.6. Thus, Nauyaca prevents of giving boundaries manually outside the default physical boundaries, which by design, have been thought to consider almost any kind of planets and orbits. 

A similar thing would happens, for example, if we set a negative lower limit in masses (which have not sense). In this case, the negative limit will be changed to 0.0123032 (M_earth).


**2.-** We did not establish boundaries for argument of periastron, mean anomaly or ascending node, and thus the default boundaries arise.



**Fixed parameters:** To keep a planetary parameter constant just set the lower and upper boundaries equal.

For example: By construction in the coordinate system, the ascending node of $at$ $least$ $one$ $planet$ in the system must be fixed.
Let's keep fixed this angle to the correct ascending node of "Planet-b", by setting equal values in the lower and upper boundaries:

In [7]:
Planetb.ascending_node = [88.36, 88.36]  # Ascending node [deg]

By doing this, we say to Nauyaca that ascending node of "Planet-b" will not be part of the parameter space explored by the Optimizers or MCMC.

Great! that's all we need to define the properties of a planet. Let's do the same for the second planet.
Let's create another planet object and do the same as before.

In [8]:
# Planet-c
Planetc = nau.SetPlanet("Planet-c")

# Load transit times
Planetc.load_ttvs("./inputs/planetc_ttvs.dat")

# Set boundaries
Planetc.mass = [.1, 50]        
Planetc.period = [11.82, 11.84]  # Assuming the period is aroung 11.83 days 
Planetc.ecc = [0, .2]
Planetc.inclination = [85, 95]
# Let's considerate just prograde solutions, i.e, +/-90° around ascending node of first planet
Planetc.ascending_node=[0, 180.]  


print(Planetc)


 =========== Planet Summary =========== 
--> Parameter -ecc- of planet -Planet-c- set to lower physical limit: 1e-06
Planet : Planet-c
  Boundaries:
    mass: (0.1, 50)  [M_earth]
    period: (11.82, 11.84)  [d]
    ecc: (1e-06, 0.2)   
    inclination: (85, 95)  [deg]
    argument: (0.0, 360)  [deg]
    mean_anomaly: (0.0, 360)  [deg]
    ascending_node: (0.0, 180.0)  [deg]
  TTVs: True


Now we have two independent Planet objects. Let's join them together in a Planetary System object.

### 1.1.2 The Planetary System object

Once we have defined our planets, we can construct a Planetary System simply defining a system name, and stellar mass and radius (in Msun and Rsun units)

In [9]:
# Let's call our system: "SystemX"
PS = nau.PlanetarySystem("SystemX", mstar=0.9132, rstar=0.8632)

Okey, but it is not a planetary systems if there are not planets! So, let's add the Planet objects created before by passing a list with the Planet objects inside:

In [10]:
# The ordering is important!! It's more intuitive if you add them according to the proximity to the star
PS.add_planets([Planetb, Planetc]) 

--> Parameter -ecc- of planet -Planet-b- set to lower physical limit: 1e-06
--> Parameter -ecc- of planet -Planet-c- set to lower physical limit: 1e-06


Alright! Now we have a Planetary System that looks like:

In [11]:
print(PS)  # print a summary of the Planetary System


 =========== Planetary System Summary =========== 
--> Parameter -ecc- of planet -Planet-b- set to lower physical limit: 1e-06
--> Parameter -ecc- of planet -Planet-c- set to lower physical limit: 1e-06

System: SystemX
Mstar: 0.9132 Msun |  Rstar: 0.8632 Rsun
Number of planets: 2
Planet information:
------
Planet: Planet-b
  Boundaries:
    mass: (0.1, 20.0)  [M_earth]
    period: (5.34, 5.36)  [d]
    ecc: (1e-06, 0.2)   
    inclination: (85, 95)  [deg]
    argument: (0.0, 360)  [deg]
    mean_anomaly: (0.0, 360)  [deg]
    ascending_node: (88.36, 88.36)  [deg]
  TTVs: True
------
Planet: Planet-c
  Boundaries:
    mass: (0.1, 50)  [M_earth]
    period: (11.82, 11.84)  [d]
    ecc: (1e-06, 0.2)   
    inclination: (85, 95)  [deg]
    argument: (0.0, 360)  [deg]
    mean_anomaly: (0.0, 360)  [deg]
    ascending_node: (0.0, 180.0)  [deg]
  TTVs: True

Simulation attributes: 
Total time of TTVs data (Ftime): 693.0 [days]
Reference epoch of the solutions (T0JD): 1.0 [JD]
Timestep of th

#### Preparing for simulations

From the summary of our Planetary System, we see at the bottom part the simulation attributes that Nauyaca automatically calculates based on the provided information of the planets. These attributes are:

* **T0JD:** It is the reference time at which all the solutions found by Nauyaca will be referenced. By default, this number is automatically calculated by rounding down the smaller transit time found in the planet ephemerids.

* **Ftime:** It defines the maximum time of the simulations. By default, it is calculated automatically by rounding up the greater central time found in the planet ephemerids. If you define a lower value than the calculated by default, then, all the central transit times greater than this new  number will be discarted (for all the planets) and the fitting will be performed until this new final time.

* **dt:** This is the timestep of the simulations. By default, it is calculated by dividing the period of the inner planet over 30, which represent the 3.33% of the orbital period of the inner planet. Note that a timestep lower than 5% of period corresponding to the internal planet is suggested to reach a good accuracy in the simulations


Let's see these attributes automatically calculated in our system:

In [13]:
print(f"T0JD: {PS.T0JD}  [days]")
print(f"Ftime: {PS.Ftime}  [days]")
print(f"dt: {PS.dt}  [days]")

T0JD: 1.0  [days]
Ftime: 693.0  [days]
dt: 0.17847543333333343  [days]


For the purposes of this tutorial, let's change these parameters and let's see again the updated attributes in the planetary system

In [16]:
# Setting the simulation attributes
PS.simulation(T0JD=0.0, Ftime=695, dt=0.15)

print(PS)


 =========== Planetary System Summary =========== 
--> Parameter -ecc- of planet -Planet-b- set to lower physical limit: 1e-06
--> Parameter -ecc- of planet -Planet-c- set to lower physical limit: 1e-06

System: SystemX
Mstar: 0.9132 Msun |  Rstar: 0.8632 Rsun
Number of planets: 2
Planet information:
------
Planet: Planet-b
  Boundaries:
    mass: (0.1, 20.0)  [M_earth]
    period: (5.34, 5.36)  [d]
    ecc: (1e-06, 0.2)   
    inclination: (85, 95)  [deg]
    argument: (0.0, 360)  [deg]
    mean_anomaly: (0.0, 360)  [deg]
    ascending_node: (88.36, 88.36)  [deg]
  TTVs: True
------
Planet: Planet-c
  Boundaries:
    mass: (0.1, 50)  [M_earth]
    period: (11.82, 11.84)  [d]
    ecc: (1e-06, 0.2)   
    inclination: (85, 95)  [deg]
    argument: (0.0, 360)  [deg]
    mean_anomaly: (0.0, 360)  [deg]
    ascending_node: (0.0, 180.0)  [deg]
  TTVs: True

Simulation attributes: 
Total time of TTVs data (Ftime): 695 [days]
Reference epoch of the solutions (T0JD): 0.0 [JD]
Timestep of the 

Nice! It appears that it is all ready to begin the simulations. But before, a **very important** step...

#### Saving the Planetary System

Once we have our Planetary System, it is very important to save the object. It can help us to avoid writting all the code above for defining Planets and the Planetary System itself, each time we want to work with it. It is also a good form to make it portable if you want to run another simulation in another computer in another time.

Nauyaca have currently two formats to save the Planetary System object:

* **pickle:**  $Advantages:$ Fast reconstruction of the Planetary object; small file size. $Disadvantages:$ machine-readable only; it is propense to fail if you try to read it with a different Nauyaca version; probably will be deprecated in upcoming versions.

* **json:** $Advantages:$ Human readable format; no conflicts with Nauyaca versions. $Disadvantages:$ Larger file size than pickle format; could fail if trying to save non-serializable objects (rare to happen in Nauyaca).


Each one of them have advantages and disadvantages. Thus, we recommend saving in both formats with the aim of having a backup of the studied planetary system.


We save our system in these formats by doing:

In [17]:
PS.save_pickle

--> Pickle file SystemX.pkl saved


In [18]:
PS.save_json

--> JSON file SystemX.json saved


which by default, will save with the name of the Planetary System object. These files are saved in the working directory.

#### In the next tutorial, we wil see how to use these files to rebuild the Planetary System defined here. See you there!

________________________

When the Planetary System is created, a .pkl file is saved. This is a python pickle file that contains the Planetary System object and all the necessary to restart TTVs simulations. This file will be useful soonly.

Let's print a summary of our Planetary system

We can access to many properties of the planetary system, for example:

In [25]:
# Bounds are in the same order as you added the planets
# FOR EACH PLANET, the order of the parameter space is:
# 0.-mass, 1.-period, 2.-eccentricity, 3.-inclination, 
# 4.-argument, 5.-mean anomaly, 6.-ascending node

print("Bounds: ", PS.bounds) 

Bounds:  [[0.1, 20.0], [33.616, 33.618], [1e-05, 0.2], [88, 90.0], [0.0, 360.0], [0.0, 360.0], [0.1, 20], [73.506, 73.508], [1e-05, 0.2], [88, 90.0], [0.0, 360.0], [0.0, 360.0], [90.0, 270.0]]


For clarity, params_names can helps to identify the whole parameter space

In [26]:
print("Boundaries:")
for i, names in enumerate(PS.params_names.split()):
    print(f"{names}: {PS.bounds[i]}" )

Boundaries:
mass1: [0.1, 20.0]
period1: [33.616, 33.618]
ecc1: [1e-05, 0.2]
inclination1: [88, 90.0]
argument1: [0.0, 360.0]
mean_anomaly1: [0.0, 360.0]
mass2: [0.1, 20]
period2: [73.506, 73.508]
ecc2: [1e-05, 0.2]
inclination2: [88, 90.0]
argument2: [0.0, 360.0]
mean_anomaly2: [0.0, 360.0]
ascending_node2: [90.0, 270.0]


If there is a planet with constant values in its parameter space, then, it will be saved as a dictionary, being the KEY the index of the parameter and the VALUE the constant.

For example, if PS.constant_params={6:180} , it means that the ascending node of the first planet is fixed to 180 deg. On the other hand, if PS.constant_params={9:0.1}, then the eccentricity of second planet is fixed to 0.1

In [28]:
# 0:mass1; 1:period1; 2:ecc1; 3:inc1; 4:argument; 5:Mean_anomaly1; 6:ascending_node1;
# 7:mass2; 8:period2; 9:ecc2; ...

print("Constant params: ", PS.constant_params)

Constant params:  {6: 180.0}


Another useful data:

In [29]:
print("Order of planets entrance: ", PS.planets_IDs)

Order of planets entrance:  {'Planet-b': 0, 'Planet-c': 1}


In [30]:
print("TTVs: \n", PS.TTVs)

TTVs: 
 {'Planet-b': {0: [21.5523131438, 0.000629, 0.000384], 1: [55.1728893405, 0.000758, 0.000739], 2: [88.7952811376, 0.000562, 0.0004], 3: [122.418924701, 0.000559, 0.000774], 4: [156.041707622, 0.000619, 0.000623], 5: [189.669905447, 0.000975, 0.000898], 6: [223.293933475, 0.00079, 0.000654], 7: [256.923266313, 0.000856, 0.00084], 8: [290.550182491, 0.000319, 0.000548], 9: [324.168450812, 0.000524, 0.000849], 10: [357.788534159, 0.000374, 0.000499], 11: [391.407056892, 0.000406, 0.000627], 12: [425.027105064, 0.000602, 0.000454], 13: [458.649125492, 0.000805, 0.000671], 14: [492.272085472, 0.000815, 0.000871], 15: [525.895441419, 0.000733, 0.00058], 16: [559.522024235, 0.000834, 0.000946], 17: [593.145545227, 0.000628, 0.000589], 18: [626.777478433, 0.000703, 0.000679], 19: [660.405076718, 0.000474, 0.000478], 20: [694.025509307, 0.000929, 0.000471], 21: [727.647086939, 0.00042, 0.000678], 22: [761.264268301, 0.000821, 0.000723], 23: [794.88325384, 0.000627, 0.000387], 24: [828.50

In [32]:
print("First transit of planet b: ", PS.TTVs["Planet-c"][0] )

First transit of planet b:  [73.2486014843, 0.000691, 0.000637]


In [33]:
print("Second term of the Loglikelihood: ", PS.second_term_logL)

Second term of the Loglikelihood:  {'Planet-b': -82.30980735748551, 'Planet-c': -35.438919859136085}


_____________________

At this point, a pickle file has been authomatically saved. This file contains the Planetary System object and its attributes (e.g., planets) and will be helpful to run TTVs simulation in another time or computer as well as for making figures.

Doing that, we are ready to beggin the TTVs fitting.

## Runing the optimizers

Optimizers are helpful to quickly explore the parameter space and search approximate solutions to the TTVs through minimizing chi square statistic.
Optimizers joint Differential Evolution and Nelder-Mead minimization methods. Firstly, Differential Evolution explores the parameter space of each planet and looks for the best solution to the TTVs. Then, Nelder-Mead takes that solution and refines it. 

To run it, jus call .run_optimizers and pass the Planetary System object, the number of solutions to look for (niter) and specify the number of cores to run in parallel (cores).

In [ ]:
# Run Optimizers. Results of this method are saved at 'OPT' plus the Planetary System name
RESULTS_opt = Optimizers.run_optimizers(PS, nsols=21, cores=7)

Results have been saved in a .opt file. All the solutions from the optimizer consist in a list, being the first element the $\chi^2$ and the rest are the planetary parameters of the first planet concatenated with planetary parameters of the next planet(s). A flag of chi square of 1e+50 could rise if the minimizers could not find a solution.

In [ ]:
# The firt 3 results (unordered by chi2)
print(RESULTS_opt[:3])

Let's plot some results:

In [ ]:
# Mass of planet b (Mass1)
m1 = [i[1] for i in RESULTS_opt]
sns.distplot(m1, kde=False, bins=5, color='blue')
plt.xlabel(r'$Mass1$')

plt.figure()
# Eccentricity of planet c (ecc2)
e2 = [i[10] for i in RESULTS_opt]
sns.distplot(e2, kde=False, bins=5, color='orange')
plt.xlabel(r'$ecc2$')

Best solution from the optimizers

In [ ]:
# We sort the results according to the chi square
sort_res = sorted(RESULTS_opt, key=lambda j: j[0])

# Let's take 'the best'
bestChi2 = sort_res[0][0]
best_planetary_params = sort_res[0][1:]

print("Best result of the optimizers:\n")
print("chi_2: ", bestChi2)
print("\n--> Planetary parameters: ", best_planetary_params)

# Splitting by number of planets in the system:
planets = [best_planetary_params[:7], best_planetary_params[7:]]
pla1 = planets[0]
pla2 = planets[1]

print("\n--> Best planetary parameters for planet1: ")
print("  ", pla1)
print("\n--> Best planetary parameters for planet2: ",)
print("  ", pla2)

##### Remember that the 7 Planetary parameters follow always the order: 

##### mass, period, eccentricity, inclination, argument, mean anomaly and ascending node

Let's plot the best 3 solutions from the optimizer and compare them with the observed TTVs. To do that, we import a useful function in the nauyaca package: plot_TTVs.

Pass the Planetary Object and a list of lists with the solutions you want to plot

In [ ]:
from plots import plot_TTVs

# Extract the best n solutions
n_sols = 3
best_n_solutions = [s[1:] for s in sort_res[:n_sols]]
#print(best_n_solutions)

# Plot the observed TTVs and the 3 solutions
plot_TTVs(PS, best_n_solutions)

# Change figure size
plt.gcf().set_size_inches(12,12)

#plt.savefig("TTVs_optimizer.png")

Ok, that does not looks like an ideal TTVs fit. Results from the optimizers provide an overall outlook of the solutions in the parameter space. To get better solutions, just increase the number of solution in Optimizers.run_optimizers. These solutions can be used to initialize walkers in the MCMC.

## Runing the MCMC

We employ Parallel-Tempering sampling algorithm to explore the parameter space at different walker temperatures. This method allows the walkers do not get stuck in local high probability regions due to the exchange between high and low temperature walkers. The posteriors are drawn from the walkers at temperature 0.

Let's define the main parameters to run the MCMC

In [ ]:
Ntemps = 10       # Number of temperatures
Nwalkers = 250    # Number of walkers per temperature
Tmax = 1e3        # Maximum temperature. If None, then Parallel-Tempering chose an appropiate maximum temperature

itmax = 1000      # Maximum number of steps
conver_steps = 10 # Number of steps to save the chains

Initial walkers should be provided in an array of shape (ntemps, nwalkers, ndim), where ndim is the number of dimensions (mass1, period1...).

We provides 3 forms of easily initialize walkers.

In [ ]:
from utils import initial_walkers

# You cand initialize walkers from the results of the optimizers
# from a previous run just typing:
#RESULTS_opt = list(np.genfromtxt('SystemX.opt'))


""" 1) Uniform distribution
 For all the planetary parameters, initialize walkers from an uniform distribution
 along the stablished boundaries for each planet"""
# Using this option is unnecesary to run optimizers previously
walkers_uniform = initial_walkers(PS, distribution="Uniform", 
                                 ntemps=Ntemps, nwalkers=Nwalkers)

"""   2) Gaussian distribution
 Provide directly the results of the optimizers (throuhgt opt_data argument) and 
 construct a gaussian with the mean and std of the solutions along each dimension."""
# In opt_data, pass RESULTS_opt (from optimizers) or results_opt (from file)
# treshold selects a fraction of the best solutions and discard the rest.
# For example, .8 takes the 80% of the solutions with the best Chi**2
walkers_gaussian = initial_walkers(PS, distribution="Gaussian", 
                                   ntemps=Ntemps, nwalkers=Nwalkers, 
                                   opt_data=RESULTS_opt, threshold=0.6)    

"""   3) Picked distribution
  Provide directly the results of the optimizers (throuhgt opt_data argument) and 
  construct a discrete distribution by randomly selecting exact values of the optimizer solutions"""
# In opt_data, pass RESULTS_opt (from optimizers) or results_opt (from file)
# treshold selects a fraction of the best solutions and discard the rest.
# For example, .8 takes the 80% of the solutions with the best Chi**2
walkers_picked = initial_walkers(PS, distribution="Picked", 
                                 ntemps=Ntemps, nwalkers=Nwalkers, 
                                 opt_data=RESULTS_opt, threshold=0.6)


Note that if there are constant values in the planet's boundaries, then that parameter will be skipped and values will not appear in the initial walkers

In [ ]:
# Comparing the initial distributions of the walkers for the dimension -dim- of temperature 0
# For dim: 0: mass1, 1: period1, ...,7: mass2, 8: period2..

dim = 7
plt.figure(figsize=(10,10))
plt.title(f"Initial distribution of dimension {dim}")
plt.hist(walkers_uniform[0,:,dim], alpha = 0.3, label='Uniform')
plt.hist(walkers_gaussian[0,:,dim], alpha = 0.3, label='Gaussian')
plt.hist(walkers_picked[0,:,dim], alpha = 0.3, label='Picked')
plt.legend()

plt.xlabel(PS.params_names.split()[dim])

Let's select the picked option to run the MCMC

In [ ]:
RESULTS_mcmc = MCMC.run_mcmc(
            PS,                           # Planetary System object
            pop0=walkers_picked,        # Initial population of walkers 
            nwalkers=Nwalkers,            # Number of walkers
            ntemps=Ntemps,                # Number of temperatures
            Tmax=Tmax,                    # Maximum temperature
            betas=None,                   # Betas is an array of inverse temperatures
            Itmax=itmax,                  # Maximum nomber of iterations (steps)
            conver_steps=conver_steps,    # Save data each this number of steps
            cores=7,                      # Number of cores to run in parallel
            suffix='')                    # Suffix to add at the final hdf5 file

Results of the MCMC are saved in a .hdf5 file. Also at the end of the iterations, best solutions are extracted (corresponding to the solutions of 'better posterior') and saved in a .best file 

In [ ]:
import h5py
f = h5py.File('SystemX.hdf5', 'r')

print(f['COL_NAMES'].value[:])

f.close()

##### MCMC Summary

Now, let's see a a summary of the results. Just provide the .hdf5 file.

In [ ]:
from utils import mcmc_summary

mcmc_summary("SystemX.hdf5")

## Visualization

There is a collection of prefabricated methods to easily visualize the results from the MCMC. 

Let's visualize the histograms of the posteriors at temperature 0

In [ ]:
from plots import plot_hist

print(RESULTS_mcmc.chain[0].shape)

# Let's pass the chains of temperature 0
# burning remove a percentage of the initial chains
plot_hist(PS, chains=RESULTS_mcmc.chain[0], burning=.2)

#plt.savefig("histograms.png")

Let's visualize the chains at temperature 0

In [ ]:
from plots import plot_chains

plot_chains(PS, chains=RESULTS_mcmc.chain[0])
plt.xlabel(f'Iterations / {conver_steps}')

#plt.savefig("chains.png")

And now, a corner plot to visualize possible correlation between parameters

In [ ]:
from plots import plot_corner

plot_corner(PS, chains=RESULTS_mcmc.chain[0], burning=0.2)
plt.gcf().set_size_inches(12,12)

#plt.savefig("corner.png")

______________________

## After simulations

At this point you can restart the kernel to simulate that we have finished the TTVs inversion and it is another time. 

Along the simulations many files have been saved automatically. Considering a Planetary System of name 'PSystem', the output files are:

    * PSystem.pkl - Contains the Planetary System object saved through pickle.
    * PSystem.opt - Contains the results of the optimizers
    * PSystem.hdf5 - Contains the main features of the MCMC
    * PSystem.best - Contains the best solutions of each saved iteration in the MCMC sorted by -chi2

Now let's see how to analyze the results saved in those files, in another moment or in a different computer.

In [ ]:
# Let's load the Planetary System
import pickle

with open('SystemX.pkl', 'rb') as input:
    PS_pkl = pickle.load(input)

print(PS_pkl)

### Result from the optimizers

In [ ]:
from plots import plot_TTVs
import matplotlib.pyplot as plt
import numpy as np

# From the file
results_file = np.genfromtxt("SystemX.opt")
results_opt = results_file.tolist()           #Convert data array to list

sort_res = sorted(results_opt, key=lambda j: j[0])

# Extract the best n solutions
n_sols = 3
best_n_solutions = [s[1:] for s in sort_res[:n_sols]]
print(best_n_solutions)

# Plot the observed TTVs and the 3 solutions
plot_TTVs(PS_pkl, best_n_solutions)

plt.gcf().set_size_inches(12,12)  # Change figure size


#plt.savefig("TTVs_optimizer.png")

### Results from the saved hdf5

Let's see how was de MCMC. plot_monitor will shou you a couple of plots to assess the performance of the MCMC

In [ ]:
"""
Upper left: This is the behavior of the temperatures. They adapt by themselves to increase
the exchange of walkers between hot and cold temperatures and viceversa. Black solid lines is for temperature 0.

Upper right: This is the percentage of walkers exchanges. Black solid lines is for temperature 0.

Lower left: This is the evolution of chi2. Blue line is the mean chi2 and orange is 'the best' chi2.

Lower right: Mean autocorrelation time. It's a measurement of the number of steps requiered to forget the initial
chain state.
"""

from plots import plot_monitor

plot_monitor("SystemX.hdf5")


Let's make a Geweke test and plot to asses for ergodicity. It takes the first 10% of the chains and divide the last 50% in 20 chunks to perform a z-test comparing the medians. If values of the test are between -2 and 2, then it is evidence of convergence.

In [ ]:
from utils import geweke

Z_test = geweke(PS_pkl, hdf5_file='SystemX.hdf5',burning=0.2)
print(Z_test[0])


In [ ]:
# Geweke plot:
plt.figure(figsize=(12,8))
for i, zt in enumerate(Z_test):
    plt.scatter(range(len(zt)), zt, label=PS_pkl.params_names.split()[i])

plt.legend(bbox_to_anchor=(1, 1), loc='upper left', ncol=1)
plt.axhline(-2, alpha=0.5, color='red')
plt.axhline(2, alpha=0.5, color='red')
plt.xlabel("Slice number")

In [ ]:
from plots import plot_hist

plot_hist(PS_pkl, chains=None, hdf5_file='SystemX.hdf5', burning=2.0)

In [ ]:
from plots import plot_chains

# By default, temperature 0 will be used
plot_chains(PS_pkl, chains=None, hdf5_file='SystemX.hdf5')

In [ ]:
from plots import plot_corner

plot_corner(PS_pkl, chains=None, hdf5_file='SystemX.hdf5', burning=0.1)
plt.gcf().set_size_inches(12,12)
plt.show()

In [ ]:
MCMC.extract_best_solutions("SystemX.hdf5")

In [ ]:
from plots import plot_TTVs
import numpy as np

# Best solutions found in the MCMC (from the file)
best_file = np.genfromtxt("SystemX.best")
results_mcmc = best_file.tolist() #Convert data array to list

# Reverse the list because we sorte by -chi**2
sort_res = sorted(results_mcmc, key=lambda j: j[0], reverse=True)

# Extract the best n solutions
n_sols = 1
print(sort_res[n_sols])
best_n_solutions = [s[1:] for s in sort_res[:n_sols]]

print(best_n_solutions)

# Plot the observed TTVs and the 3 solutions
plot_TTVs(PS_pkl, best_n_solutions)

plt.gcf().set_size_inches(12,12)  # Change figure size

## Extra features

Suposse you have a set of parameters and you want to calculate the central transit times produced by two planets along 300 days of observations. Well, this is how you can comunicate with TTVFast:

In [ ]:
from utils import run_TTVFast

set_of_params = [5.33031,  33.62651, 0.06558, 88.47247, 202.35109, 246.71734, 170.5195,
                 11.60434, 73.48983, 0.19693, 88.84697, 61.90508,  121.72177, 180.0]

transits = run_TTVFast(set_of_params, mstar=0.91, NPLA=2, Tin=0., Ftime=300)

print(transits)

First list is the planet ID corresponding to the entrance order. Second list is the transit epochs. Third list are the central transit times. To facilitate the reading of the transits, let's provide a Planetary System object and the transits.

In [ ]:
from utils import calculate_epochs

epochs = calculate_epochs(transits, PS_pkl)

print(epochs)

Finally, let's calculate the chi2 of the set of parameters. Note that we removed the constant parameters from the set_of_params. It's because PS_pkl contains that information.

In [ ]:
from utils import calculate_chi2

print("Constants: ", PS_pkl.constant_params)

incomplete_params = [5.33031,  33.62651, 0.06558, 88.47247, 202.35109, 246.71734, 170.5195,
                     11.60434, 73.48983, 0.19693, 88.84697, 61.90508,  121.72177 ] # 180.0]

chi2 = calculate_chi2(incomplete_params, PS_pkl)
print("chi2: ", chi2)

### Additional notes:

* Sometimes there will raise an "Invalid proposal" legend. It just indicates that calculate_epochs could not determine transits due to non-transiting planets, planetary ejection, etc. Don't worry about that except if that's the only notification you see. In that case, consider to inspect the planetary boundaries.



## The End